# FRETBursts Paper Main Notebook

*This the **main notebook** for reproducing code examples and figures for the paper:*

<br>
<div class="alert alert-success">
**FRETBursts: Open Source Burst Analysis Toolkit for Confocal Single-Molecule FRET**<br>
Ingargiola et al. (2016). biorXiv doi:[10.1101/039198](http://dx.doi.org/10.1101/039198)
</div>

# 0. Initializations

## Loading the software

Load software and define default figure style.

In [ ]:
from fretbursts import *
import phconvert as phc
print('phconvert: ', phc.__version__)

In [ ]:
sns = init_notebook(fs=13, savefig_dpi=75)

In [ ]:
figure_size = (5, 4)
default_figure = lambda: plt.subplots(figsize=figure_size)
save_figures = True

def savefig(filename, **kwargs):
    if not save_figures:
        return
    import os
    name, extension = os.path.splitext(os.path.basename(filename))
    dir_ = '../figures/%s/' % name
    kwargs_ = dict(dpi=150, bbox_inches='tight', 
                   frameon=True, facecolor='white', transparent=False)
    kwargs_.update(kwargs)
    plt.savefig(dir_ + name + extension, **kwargs_)

## Getting the data

Download the data file.

In [ ]:
url = 'http://files.figshare.com/2182601/0023uLRpitc_NTP_20dT_0.5GndCl.hdf5'
download_file(url, save_dir='./data')

In [ ]:
filename = './data/' + url.split('/')[-1]
filename

# Section 1. Introduction

No code for this section.

# Section 2. FRETBursts Overview

No code for this section.

# Section 3. Architecture and Concepts

## 3.1 Photon Streams

No code of figures in this section.

## 3.2 Background Definitions

To reproduce **Fig. 2 & 3** see the notebook:

- [Figures - Background plots](Figures - Background plots.ipynb)

## 3.3 The Data Class

No code for this section.

## 3.4 Introduction to Burst Search

No code for this section.

## 3.5 γ-corrected Burst Sizes and Weights

No code for this section.

## 3.6 Plotting Data

This section of the paper describes the general syntax for plotting in FRETBursts.
For practical reasons (i.e. we need to load the data first),
these plot examples are reproduced later in this notebook.

To avoid errors, it is recommended 
to run the notebook sequentially, without jumping section.

# Section 4. smFRET Burst Analysis

## 4.1 Loading the Data

In [ ]:
d = loader.photon_hdf5(filename)

## 4.2 Alternation Parameters

In [ ]:
d.add(D_ON=(2100, 3900), A_ON=(100, 1900))

In [ ]:
bpl.plot_alternation_hist(d)

In [ ]:
loader.usalex_apply_period(d)

For reproducing the alternation figure (**Fig. 1**) see the notebook:

- [Figures - ALEX histograms](Figures - ALEX histograms.ipynb)

For additional information, see the *μs-ALEX parameters* section in FRETBursts notebook:

- [FRETBursts - us-ALEX smFRET burst analysis](http://nbviewer.ipython.org/urls/raw.github.com/tritemio/FRETBursts_notebooks/master/notebooks/FRETBursts%2520-%2520us-ALEX%2520smFRET%2520burst%2520analysis.ipynb)

## Plot examples from section 3.6

In this sections there are some examples on how to make plot with FRETBursts.

To plot a timetrace of the measurement we use:

In [ ]:
dplot(d, timetrace);

Here, `dplot` is a generic wrapper (the same for all plots)
that takes care of setting up the figure, title and axis 
(in the multispot case `dplot` creates multi-panel plot).
The second argument, `timetrace`, is the actual plot function. 
All the eventual additional arguments passed to `dplot` are, 
in turn, passed to the plot function (e.g. `timetrace`).

If we look at the documentation for [`timetrace`](http://fretbursts.readthedocs.org/en/latest/plots.html#fretbursts.burst_plot.timetrace)
function we notice that it accepts a long list of arguments.
In python, when an argument is not specified, it will take the default
value specified in the function definition (see previus link).

As an example, to change the bin size (i.e. duration) of the timetrace histogram,
we can look up in the [`timetrace` documentation](http://fretbursts.readthedocs.org/en/latest/plots.html#fretbursts.burst_plot.timetrace)
and find that the argument we need to modify is `binwidth`
(we can also see that the default value is `0.001` seconds).
We can then re-plot the timetrace using a bin of 0.5 ms:

In [ ]:
dplot(d, timetrace, binwidth=0.5e-3);

> ### See also:
>
> - [bpl.timetrace](http://fretbursts.readthedocs.org/en/latest/plots.html#fretbursts.burst_plot.timetrace)
>    function documentation
> - [bpl.ratetrace](http://fretbursts.readthedocs.org/en/latest/plots.html#fretbursts.burst_plot.ratetrace)
>   function documentation
> - [Plotting basics](http://nbviewer.jupyter.org/github/tritemio/FRETBursts_notebooks/blob/master/notebooks/FRETBursts%20-%20us-ALEX%20smFRET%20burst%20analysis.ipynb#Plotting-basics) in FRETBursts us-ALEX tutorial.

## 4.3 Background Estimation

Background estimation using a threshold of 2ms and a background period of 30s:

In [ ]:
d.calc_bg(bg.exp_fit, time_s=30, tail_min_us=2000)

Background estimation automatic threshold and a background period of 20s:

In [ ]:
d.calc_bg(bg.exp_fit, time_s=20, tail_min_us='auto', F_bg=1.7)

> For more info see [Background estimation](http://nbviewer.jupyter.org/github/tritemio/FRETBursts_notebooks/blob/master/notebooks/FRETBursts%20-%20us-ALEX%20smFRET%20burst%20analysis.ipynb#Background-estimation) section in the us-ALEX tutorial.

In [ ]:
dplot(d, hist_bg, period=0);

In [ ]:
dplot(d, timetrace_bg);

## 4.4 Burst Search

### 4.4.1 Burst Search in FRETBursts

This is a sliding-window burst search using all-photons stream,
m=10 photons to compute the rate, and a threshold for burst
start 6 times larger (`F`) than the local background rate:

In [ ]:
d.burst_search(F=6, m=10, ph_sel=Ph_sel('all'))

In the next example, we fix the burst search rate threshold to a constant value of 50kcps:

In [ ]:
d.burst_search(min_rate_cps=50e3, m=10, ph_sel=Ph_sel('all'))

Here, we perform a dual-channel burst search (DCBS):

In [ ]:
d_dcbs = bext.burst_search_and_gate(d, F=6, m=10)

> ### Related documentation:
>
> - [Data.burst_search](http://fretbursts.readthedocs.org/en/latest/data_class.html?highlight=burst_search#fretbursts.burstlib.Data.burst_search)
> - [bext.burst_search_and_gate](http://fretbursts.readthedocs.org/en/latest/plugins.html?highlight=burst_search_and_gate#fretbursts.burstlib_ext.burst_search_and_gate)

Here, we re-perform the initial burst search:

In [ ]:
d.burst_search(F=6, m=10, ph_sel=Ph_sel('all'))

### 4.4.2 Correction Coefficients

At any time during the analysis, setting the correction coefficients 
will automatically correct and update all the burst data:

In [ ]:
d.gamma = 0.85
d.leakage = 0.04
d.dir_ex = 0.08

## 4.5 Burst Selection

Select bursts with at least 30 photons in Dex stream:

In [ ]:
ds = d.select_bursts(select_bursts.size, th1=30)

Select bursts with *E* in the range $[0, 0.5]$ range and *S* in the range $[0.2, 0.8]$:

In [ ]:
ds = d.select_bursts(select_bursts.ES, E1=0, E2=0.5, S1=0.2, S2=0.8)

Select bursts with duration between 0.5 and 2 milliseconds:

In [ ]:
ds = d.select_bursts(select_bursts.width, th1=0.5, th2=2)

Example of burst selection combining two criteria (burst size and duration):

In [ ]:
ds = d.select_bursts(select_bursts.size, th1=50, th2=200)
dsw = ds.select_bursts(select_bursts.width, th1=0.5e-3, th2=3e-3)

### 4.5.1 Burst Size Selection

Select bursts with size of at least 30, counting both Dex and Aex photons:

In [ ]:
ds = d.select_bursts(select_bursts.size, th1=30, add_naa=True)

Select bursts with size of at least 15 using only Dex photons. 
The size is computed as:

$$ n_t = \gamma n_d + n_a $$

In [ ]:
ds = d.select_bursts(select_bursts.size, th1=15, gamma=0.65)

Finally, to get the arry of burst sizes (optionally adding `naa`) and applying
corrections we use

In [ ]:
ds.burst_sizes()

> ### Related documentation:
>
> - [select_bursts.size](http://fretbursts.readthedocs.org/en/latest/burst_selection.html#fretbursts.select_bursts.size)
> - [Data.burst_sizes](http://fretbursts.readthedocs.org/en/latest/data_class.html#fretbursts.burstlib.Data.burst_sizes)
> - [Data.burst_sizes_ich](http://fretbursts.readthedocs.org/en/latest/data_class.html#fretbursts.burstlib.Data.burst_sizes_ich)

### 4.5.2 Select the FRET Populations

A convenient way to select the FRET populuation is applying two filters.
The first, selecting bursts sizes (using Dex photons) larger than a threshold.
The second, selection bursts with `naa` (counts in the AexAem stream)
larger than a threshold:

In [ ]:
ds1 = d.select_bursts(select_bursts.size, th1=15)
ds2 = ds1.select_bursts(select_bursts.naa, th1=15)

In [ ]:
dds1 = d_dcbs.select_bursts(select_bursts.size, th1=15)
dds2 = dds1.select_bursts(select_bursts.naa, th1=15)

The result of this filter (`ds2`) can be compared with a
single burst size filter using all-photons (`ds`) in the plots below:

In [ ]:
alex_jointplot(ds)
alex_jointplot(ds2);
alex_jointplot(dds2);

## 4.6 Population Analysis

In [ ]:
bext.bursts_fitter(ds2, 'E', binwidth=0.03,
                   model=mfit.factory_two_gaussians());

In [ ]:
fit_res = ds2.E_fitter.fit_res[0]
print(fit_res.fit_report())

In [ ]:
dplot(ds2, hist_fret, show_model=True);

The reduced $\chi^2$ can be obtained as follows:

In [ ]:
ds2.E_fitter.fit_res[0].redchi

Finally, to reproduce **Fig. 6**, we replot the previous histogram tweaking the style:

In [ ]:
dds1 = d_dcbs.select_bursts(select_bursts.size, th1=30)
dds2 = dds1.select_bursts(select_bursts.naa, th1=15)

In [ ]:
bext.bursts_fitter(dds2, 'E', binwidth=0.03,
                   model=mfit.factory_two_gaussians());

In [ ]:
fig, ax = default_figure()

cmap = 'Blues' #'Spectral_r'
c = sns.color_palette(cmap, 12)[-5]

dplot(dds2, hist_fret, show_model=True, ax=ax, 
      hist_bar_style=dict(facecolor=c, edgecolor='k', linewidth=0.3))
ax.set_title('')
ax.set_xlim(0, 1.1)
savefig('hist_fit')

# 5 Implementing Burst Variance Analysis

For this section of the paper see the dedicated notebook:

- [Figures - Burst Variance Analysis](Figures - Burst Variance Analysis.ipynb)